### Importando Bibliotecas e Pacotes

In [3]:
import numpy as np
import pandas as pd
import csv
import json
from sklearn.feature_extraction.text import CountVectorizer
import requests
import io

### Acessando os dados 

In [4]:
#(0) Faço a chamada do arquivo csv a ser analisado, (1) crio um DataFrame(df) renomeando as colunas já existentes
exp22 = requests.get('https://storage.googleapis.com/comerciobr2122/EXP_2022_MUN.csv')

dadosexp22 = pd.read_csv(io.StringIO(exp22.text), names = ['Ano', 'Mes', 'Produtos', 'Pais(dest)', 'Estado','Municipio','KG_Liq', 'Total_em_Dólar'],sep = ';',encoding="utf-8-sig", low_memory=True)

/tmp/ipykernel_9333/2036215297.py:4: DtypeWarning: Columns (0,1,2,3,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  dadosexp22 = pd.read_csv(io.StringIO(exp22.text), names = ['Ano', 'Mes', 'Produtos', 'Pais(dest)', 'Estado','Municipio','KG_Liq', 'Total_em_Dólar'],sep = ';',encoding="utf-8-sig", low_memory=True)


### Acessando as Nomeclaturas originais

In [3]:
# Realizando a leitura dos arquivos já convertidos e carregando em uma variável
with open("meses.json", "r") as jsonMes:
    meses = json.loads(jsonMes.read())
with open("produtos.json", "r") as jsonProdutos:
    produtos = json.loads(jsonProdutos.read())
with open("paises.json", "r") as jsonPais:
    paises = json.loads(jsonPais.read())
with open("estadosBr.json", "r") as jsonEstados:
    estados = json.loads(jsonEstados.read())
with open("municipios.json", "r") as jsonMun:
    municipios = json.loads(jsonMun.read())

### Limpeza e Tratamento dos Dados 

In [4]:
# Altero o tipo de dados das colunas para utilização com o map
dadosexp22["Mes"] = dadosexp22["Mes"].astype('string')
dadosexp22["Produtos"] = dadosexp22["Produtos"].astype('string')
dadosexp22["Pais(dest)"] = dadosexp22["Pais(dest)"].astype('string')
dadosexp22["Estado"] = dadosexp22["Estado"].astype('string')
dadosexp22["Municipio"] = dadosexp22["Municipio"].astype('string')

In [5]:
# Com a função drop removo a coluna "ano" e a linha "0".
dadosexp22.drop(["Ano"], axis=1, inplace=True)
dadosexp22.drop([0], inplace=True)

# Por fim faço a remoção de duplicatas e valores NA
dadosexp22 = dadosexp22.drop_duplicates()
dadosexp22 = dadosexp22.dropna()

In [6]:
# Renomeio todas as colunas abaixo utilizando como valor de saída as Nomeclaturas originais
dadosexp22["Mes"] = dadosexp22["Mes"].map(meses,na_action='ignore')
dadosexp22["Produtos"] = dadosexp22["Produtos"].map(produtos,na_action='ignore')
dadosexp22["Pais(dest)"] = dadosexp22["Pais(dest)"].map(paises,na_action='ignore')
dadosexp22["Estado"] = dadosexp22["Estado"].map(estados,na_action='ignore')
dadosexp22["Municipio"] = dadosexp22["Municipio"].map(municipios,na_action='ignore')

In [7]:
# Seleciono o período a ser analisado com a função loc (Jan,Fev,Mar,Abril,Maio,Junho) 
dadosexp22 = dadosexp22.loc[dadosexp22['Mes'].isin(['Janeiro','Fevereiro','Março',
                                              'Abril','Maio','Junho','Julho',
                                              'Agosto','Setembro','Outubro',
                                              'Novembro','Dezembro'])]

In [8]:
# Incluo o 'index' do pandas como propriedade do DF e reordeno o DF
dadosexp22["Id_Venda"] = dadosexp22.index
dadosexp22 = dadosexp22[['Id_Venda','Municipio','Estado','Pais(dest)','Produtos',
                     'Total_em_Dólar','KG_Liq','Mes']]

In [9]:
dadosexp22.head(8)

,Id_Venda,Municipio,Estado,Pais(dest),Produtos,Total_em_Dólar,KG_Liq,Mes
1,1,Ampére,Paraná,Uruguai,Outros móveis e suas partes,47615,53482,Abril
3,3,Vitória,Espírito Santo,Panamá,"Massas alimentícias, mesmo cozidas ou recheada...",2264,564,Setembro
4,4,São José,Santa Catarina,Argentina,"Camiseiros, blusas, blusas-camiseiros, de uso ...",6642,85,Janeiro
5,5,Jaraguá do Sul,Santa Catarina,Estados Unidos,"Motores e geradores, elétricos, exceto os grup...",12155328,2942192,Fevereiro
7,7,Vitória,Espírito Santo,Bahamas,"Leitelho, leite e nata coalhados, iogurte, kef...",100,32,Maio
8,8,Ribeirão Preto,São Paulo,Argentina,Aparelhos de raios X e aparelhos que utilizem ...,217635,5234,Novembro
9,9,Ribeirão das Neves,Minas Gerais,Argentina,"Acessórios para tubos [por exemplo: uniões, co...",1092,31,Agosto
11,11,Palhoça,Santa Catarina,Sérvia,"Fatos de saia-casaco, conjuntos, casacos, vest...",1935,11,Junho


> Informações sobre a receita 'bruta' total obtida com as Exportações Brasileiras (Brasil -> Comércio Exterior) e o total de operações.

In [10]:
# Altero o tipo das colunas que serão utilizadas
dadosexp22['Total_em_Dólar'] = dadosexp22['Total_em_Dólar'].astype('int')

In [11]:
# Calculando os valores das importações 2022
expor_total22 = dadosexp22["Total_em_Dólar"].sum()
expor_count22 = dadosexp22["Total_em_Dólar"].count()
expor_media22 = dadosexp22["Total_em_Dólar"].mean()

In [12]:
# Crio um dataframe utilizando o "pandas"
exportacoes22 = pd.DataFrame({"Ano": ["2022"],
                              "Total_de_Transacoes": expor_count22,
                              "Receita_Total_Exportacoes": expor_total22,
                              "Valor_medio_das_Exportacoes": expor_media22})
# Incluo uma coluna no meu DF com passando o 'Index'
exportacoes22["exportacoes22_id"] = exportacoes22.index

In [13]:
# Edito a formatação numérica das colunas 
exportacoes22 ["Total_de_Transacoes"] = exportacoes22["Total_de_Transacoes"].map("{:,.0f}".format)
exportacoes22 ["Receita_Total_Exportacoes"] = exportacoes22["Receita_Total_Exportacoes"].map("${:,.0f}".format)
exportacoes22 ["Valor_medio_das_Exportacoes"] = exportacoes22["Valor_medio_das_Exportacoes"].map("${:,.0f}".format)

In [14]:
# Faço uma 'Reordenação' do DF
exportacoes22 = exportacoes22[['exportacoes22_id','Ano','Total_de_Transacoes',
                               'Receita_Total_Exportacoes','Valor_medio_das_Exportacoes']]

In [15]:
exportacoes22

,exportacoes22_id,Ano,Total_de_Transacoes,Receita_Total_Exportacoes,Valor_medio_das_Exportacoes
0,0,2022,"994,112","$307,283,561,759","$309,104"


> Faço um resumo, individual, da receita 'bruta' obtida com as Exportações por cada Estado Brasileiro.

In [16]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
uf_total22 = dadosexp22.groupby('Estado')['Total_em_Dólar'].sum()
uf_media22 = dadosexp22.groupby("Estado")["Total_em_Dólar"].mean()
uf_operacoes22 = dadosexp22.groupby("Estado")["Total_em_Dólar"].count()

In [17]:
# Crio um DataFrame para armazenar os Valores agrupados
uf_analises22 = pd.DataFrame({"Numero_de_Transacoes" : uf_operacoes22,
                              "Valor_medio_das_Exportacoes" : uf_media22,
                              "Valor_total_das_Exportacoes" : uf_total22})

#  Incluo uma coluna no meu DF com passando o 'Index'
uf_analises22["Uf_analises22_id"] = uf_analises22.index

In [18]:
# Edito a formatação numérica dos dados
uf_analises22 = uf_analises22.round(2)
uf_analises22 ["Valor_medio_das_Exportacoes"] = uf_analises22["Valor_medio_das_Exportacoes"].map("${:,.2f}".format)
uf_analises22 ["Valor_total_das_Exportacoes"] = uf_analises22["Valor_total_das_Exportacoes"].map("${:,.2f}".format)

In [19]:
# Faço uma 'Reordenação' do DF
uf_analises22 = uf_analises22[['Uf_analises22_id','Numero_de_Transacoes','Valor_total_das_Exportacoes',
                               'Valor_medio_das_Exportacoes']]

In [20]:
uf_analises22

,Uf_analises22_id,Numero_de_Transacoes,Valor_total_das_Exportacoes,Valor_medio_das_Exportacoes
Estado,,,,
Acre,Acre,1036,"$38,419,535.00","$37,084.49"
Alagoas,Alagoas,6910,"$383,458,865.00","$55,493.32"
Amapá,Amapá,2409,"$178,741,199.00","$74,197.26"
Amazonas,Amazonas,7586,"$1,029,263,696.00","$135,679.37"
Bahia,Bahia,19338,"$12,631,785,259.00","$653,210.53"
Ceará,Ceará,13609,"$2,136,946,280.00","$157,024.49"
Distrito Federal,Distrito Federal,977,"$117,350,618.00","$120,113.22"
Espírito Santo,Espírito Santo,30438,"$8,601,362,892.00","$282,586.34"
Exterior,Exterior,24,"$274,992.00","$11,458.00"


> Detalhamento individual dos valores arrecadados por cada Munícipio/Sede das Empresas Exportadoras.

In [21]:
# Inicio agrupando os valores/operações de cada estado e realizo os devidos calculos
mun_total22 = dadosexp22.groupby("Municipio")["Total_em_Dólar"].sum()
mun_media22 = dadosexp22.groupby("Municipio")["Total_em_Dólar"].mean()
mun_operacoes22 = dadosexp22.groupby("Municipio")["Total_em_Dólar"].count()

In [22]:
# Criando um DataFrame dos Municípios com os valores agrupados
mun_analises22 = pd.DataFrame({"Numero_de_Vendas_para_o_Exterior" : mun_operacoes22,
                               "Valor_medio_das_Exportacoes" : mun_media22,
                               "Valor_total_das_Exportacoes" : mun_total22})


#  Incluo uma coluna no meu DF com passando o 'Index'
mun_analises22["Mun_analises22_id"] = mun_analises22.index

In [23]:
# Edito a formatação numérica dos dados
mun_analises22 ["Valor_total_das_Exportacoes"] = mun_analises22["Valor_total_das_Exportacoes"].map("${:,.0f}".format)
mun_analises22 ["Valor_medio_das_Exportacoes"] = mun_analises22["Valor_medio_das_Exportacoes"].map("${:,.0f}".format)

In [24]:
# Faço uma 'Reordenação' do DF
mun_analises22 = mun_analises22[["Mun_analises22_id","Numero_de_Vendas_para_o_Exterior",
                                 "Valor_total_das_Exportacoes", "Valor_medio_das_Exportacoes"]]

In [25]:
mun_analises22.sort_values(by="Numero_de_Vendas_para_o_Exterior", ascending=False).head(8)

,Mun_analises22_id,Numero_de_Vendas_para_o_Exterior,Valor_total_das_Exportacoes,Valor_medio_das_Exportacoes
Municipio,,,,
Santos,Santos,43036,"$4,366,825,933","$101,469"
São Paulo,São Paulo,40041,"$4,044,612,427","$101,012"
Sorocaba,Sorocaba,22010,"$1,997,408,697","$90,750"
Rio de Janeiro,Rio de Janeiro,21469,"$19,862,874,716","$925,189"
Campinas,Campinas,18223,"$931,524,655","$51,118"
Paranaguá,Paranaguá,17607,"$5,630,503,485","$319,788"
Jundiaí,Jundiaí,16683,"$646,120,151","$38,729"
Guarulhos,Guarulhos,15878,"$2,013,724,472","$126,825"


> Principais destinos das Exportações Brasileiras. 

In [26]:
model_22 = dadosexp22["Pais(dest)"]
model_22.index = range(0, 994112, 1)

In [27]:
model_22

0                Uruguai
1                 Panamá
2              Argentina
3         Estados Unidos
4                Bahamas
               ...      
994107        Luxemburgo
994108          Paraguai
994109           Equador
994110             Suíça
994111               NaN
Name: Pais(dest), Length: 994112, dtype: object

In [28]:
destinos_22 = [str(i) for i in model_22]

In [29]:
# Criando um vetor
vetor_destinos22 = CountVectorizer(analyzer = 'word', lowercase=False)

In [30]:
# Vetorizando o dataset
destinos22 = vetor_destinos22.fit_transform(destinos_22)

In [31]:
# Retornando paises únicos
destinos_unicos22 = vetor_destinos22.get_feature_names_out()

In [32]:
# Criar um novo DataFrame dos Paises
dfdestinos22 = pd.DataFrame(destinos22.todense(), columns = destinos_unicos22, index = model_22.index)

In [33]:
# Calcular o percentual 
porcent_destinos22 = 100 * pd.Series(dfdestinos22.sum()).sort_values(ascending = False) / dfdestinos22.shape[0]

In [34]:
# Faço a 'limpeza' de 'Dados' reconhecidos com 'nan'
porcent_destinos22.drop(["nan"], inplace=True)

In [35]:
# Crio um DF para melhor aproveitamento dos Dados
porcentagem_dest22 = pd.DataFrame({'Destinos22':porcent_destinos22})

# Incluo o 'Index' como propriedade do DF
porcentagem_dest22['Destinos22_id'] = porcentagem_dest22.index

# Reordeno o DF
porcentagem_dest22 = porcentagem_dest22[['Destinos22_id','Destinos22']]

In [36]:
# Um head dos 5 primeiros 
porcentagem_dest22.head(10)

,Destinos22_id,Destinos22
Unidos,Unidos,7.761097
Estados,Estados,7.030596
Paraguai,Paraguai,6.608410
Uruguai,Uruguai,4.832152
Chile,Chile,4.513375
Colômbia,Colômbia,3.728151
Peru,Peru,3.154675
México,México,2.979242
Panamá,Panamá,2.800590
Ilhas,Ilhas,2.451836


> Abaixo localizo os produtos mais requisitados por importadoras no Mercado Exterior.

In [37]:
# Agrupo os Produtos e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
produtos_contagem22 = dadosexp22.groupby("Produtos")["Total_em_Dólar"].count()
produtos_soma22 = dadosexp22.groupby("Produtos")["Total_em_Dólar"].sum()

In [38]:
# Crio um DF com os valores acima
produtosunicos22 = pd.DataFrame({"Quantidade_de_Exportacoes" : produtos_contagem22,
                                 "Valor_total_das_Exportacoes" : produtos_soma22})

# Incluo o 'Index no meu DF'
produtosunicos22['ProdutosUnicos22_id'] = produtosunicos22.index

In [39]:
# Faço uma pequena alteração na saída 
produtosunicos22["Valor_total_das_Exportacoes"] = produtosunicos22["Valor_total_das_Exportacoes"].map("${:,.0f}".format)

In [40]:
# Reordeno as 'Colunas' do meu DF
produtosunicos22 = produtosunicos22[['ProdutosUnicos22_id',
                                     'Quantidade_de_Exportacoes','Valor_total_das_Exportacoes']]

In [41]:
produtosunicos22.head()

,ProdutosUnicos22_id,Quantidade_de_Exportacoes,Valor_total_das_Exportacoes
Produtos,,,
"Abrasivos naturais ou artificiais, em pó ou em grãos, aplicados sobre matérias têxteis, papel, cartão ou outras matérias, mesmo recortados, costurados ou reunidos de outro modo","Abrasivos naturais ou artificiais, em pó ou em...",1690,"$34,452,320"
"Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria","Absorventes e tampões higiênicos, cueiros e fr...",513,"$40,703,655"
"Acessórios para tubos (por exemplo, uniões, cotovelos, luvas), de alumínio","Acessórios para tubos (por exemplo, uniões, co...",716,"$4,499,438"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de cobre","Acessórios para tubos [por exemplo: uniões, co...",1609,"$4,722,830"
"Acessórios para tubos [por exemplo: uniões, cotovelos, mangas (luvas)], de ferro fundido, ferro ou aço","Acessórios para tubos [por exemplo: uniões, co...",3831,"$84,364,312"


> Busco localizar os Meses com maiores 'picos' de vendas para o Exterior 

In [42]:
# Agrupo os Meses e realizo a sua Contagem, em seguida, Somo os Valores de todas as suas Compras
meses_contagem22 = dadosexp22.groupby("Mes")["Total_em_Dólar"].count()
meses_soma22 = dadosexp22.groupby("Mes")["Total_em_Dólar"].sum()

In [43]:
# Crio um DF com os valores acima
principal_mes22 = pd.DataFrame({"Quantidade_de_Exportacoes_no_Mes" : meses_contagem22,
                                 "Valor_total_das_Exportacoes_no_Mes" : meses_soma22})

# Incluo o 'Index no meu DF'
principal_mes22['principal_mes22_id'] = principal_mes22.index

In [44]:
# Faço uma pequena alteração na saída 
principal_mes22["Valor_total_das_Exportacoes_no_Mes"] = principal_mes22["Valor_total_das_Exportacoes_no_Mes"].map("${:,.0f}".format)

In [45]:
# Reordeno as 'Colunas' do meu DF
principal_mes22 = principal_mes22[['principal_mes22_id',
                                   'Quantidade_de_Exportacoes_no_Mes','Valor_total_das_Exportacoes_no_Mes']]

In [46]:
principal_mes22.sort_values(by='Valor_total_das_Exportacoes_no_Mes', ascending=False).head(8)

,principal_mes22_id,Quantidade_de_Exportacoes_no_Mes,Valor_total_das_Exportacoes_no_Mes
Mes,,,
Junho,Junho,93860,"$32,742,194,543"
Agosto,Agosto,95306,"$30,786,908,834"
Julho,Julho,91701,"$29,844,324,640"
Maio,Maio,95675,"$29,642,851,197"
Março,Março,91178,"$29,422,320,196"
Abril,Abril,87694,"$28,969,694,687"
Setembro,Setembro,93166,"$28,587,334,142"
Novembro,Novembro,88488,"$27,652,303,451"


#### Feito por mim, <a href="https://linkedin.com/in/mario-barcelos"> Mario Barcelos! </a> 